<a href="https://colab.research.google.com/github/merrcah/TinyBasic/blob/main/Copy_of_asg2_expressions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%mkdir "ply"
%cd ply
!wget "https://raw.githubusercontent.com/dabeaz/ply/master/src/ply/lex.py"
!wget "https://raw.githubusercontent.com/dabeaz/ply/master/src/ply/yacc.py"

__file__ = "asg2_expressions.ipynb"

# -----------------------------------------------------------------------------
# example.py
#
# Example of using PLY To parse the following simple grammar.
#
#   EXAMPLE VALID PROGRAM: 
#      x=3*4;y=4-1;y+3
#      x=3*4;y=4-1;3+y
#      4
#      4+5
#      4+(5+(6+7))
#
#   LONGER EXAMPLE PROGRAM
#   PROGRAM==>assignment_list expression
#          ==>assignment ; assignment ; assignment; assignment; expression
#          ==>let NAME = expression ; assignment ; assignment; assignment; expression
#          let w=5; let x=3; let y=4+x; let z=7; 3+x*y
#   EXAMPLE INVALID PROGRAM: 
#      let x=3; let y=4; 3+x*y; let z=3; (you can't have assignment statements at the end)
#      4+5+7 (you can't chain the plus operation)
#
#   CONTEXT FREE GRAMMAR:
#
#   program : [assignment_list] expression
#
#   assginment_list : assignment ;
#              | assignment ; assignment_list
#
#   assignment : NAME = expression
# 
#   expression : term PLUS term
#              | term MINUS term
#              | term
#
#   term       : factor TIMES factor
#              | factor DIVIDE factor
#              | factor
#
#   factor     : NUMBER
#              | NAME
#              | PLUS factor
#              | MINUS factor
#              | LPAREN expression RPAREN
#
# -----------------------------------------------------------------------------

from ply.lex import lex
from ply.yacc import yacc  

# --- Tokenizer

# All tokens must be named in advance.
tokens = ( 'LET', 'PLUS', 'MINUS', 'TIMES', 'DIVIDE', 'LPAREN', 'RPAREN',
           'NAME', 'NUMBER', 'EQUALS', 'SEMICOLON' )

# Ignored characters
t_ignore = ' \t'

# Token matching rules are written as regexs
t_PLUS = r'\+'
t_MINUS = r'-'
t_TIMES = r'\*'
t_DIVIDE = r'/'
t_LPAREN = r'\('
t_RPAREN = r'\)'
t_EQUALS = r'='
t_SEMICOLON = r';'

# A function can be used if there is an associated action.
# Write the matching regex in the docstring.
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t
def t_NAME(t):
    r'[a-zA-Z_][a-zA-Z0-9_]*'
    # scan through all the reserved words and update the type
    if t.value == 'let' or t.value == 'LET':
        t.type = 'LET'
    return t

# Ignored token with an action associated with it
def t_ignore_newline(t):
    r'\n+'
    t.lexer.lineno += t.value.count('\n')

# Error handler for illegal characters
def t_error(t):
    print(f'Illegal character {t.value[0]!r}')
    t.lexer.skip(1)

# Build the lexer object
lexer = lex()
    
# --- Parser

# Write functions for each grammar rule which is
# specified in the docstring.

def p_program_withassignments(p):
    '''
    program : assignment_list expression
    '''
    p[0] = ('assignments',p[1], p[2])

def p_program_noassignments(p):
    '''
    program : expression
    '''
    p[0] = p[1]
    
def p_assignment_listsingle(p):
    '''
    assignment_list : assignment SEMICOLON
    '''
    p[0] = (p[1])
    
def p_assignment_listmultiple(p):
    '''
    assignment_list : assignment SEMICOLON assignment_list
    '''
    p[0] = p[1] + p[3]
    
def p_assignment(p):
    '''
    assignment : LET NAME EQUALS expression
    '''
    p[0] = (p[2], p[4]) # this returns a tuple with variable name first, followed by the expression used to initialize the variable

def p_expression(p):
    '''
    expression : term PLUS term
               | term MINUS term
    '''
    # p is a sequence that represents rule contents.
    #
    # expression : term PLUS term
    #   p[0]     : p[1] p[2] p[3]
    # 
    p[0] = ('binop', p[2], p[1], p[3])

def p_expression_term(p):
    '''
    expression : term
    '''
    p[0] = p[1]

def p_term(p):
    '''
    term : factor TIMES factor
         | factor DIVIDE factor
    '''
    p[0] = ('binop', p[2], p[1], p[3])

def p_term_factor(p):
    '''
    term : factor
    '''
    p[0] = p[1]

def p_factor_name(p):
    '''
    factor : NAME
    '''
    p[0] = ('name', p[1])

def p_factor_number(p):
    '''
    factor : NUMBER
    '''
    p[0] = ('number', p[1])

def p_factor_unary(p):
    '''
    factor : PLUS factor
           | MINUS factor
    '''
    p[0] = ('unary', p[1], p[2])

def p_factor_grouped(p):
    '''
    factor : LPAREN expression RPAREN
    '''
    p[0] = ('grouped', p[2])

def p_error(p):
    print(f'Syntax error at {p.value!r}')

# Build the parser
parser = yacc()

# Parse an expression
#programstr = 'let w=5; let x=3; let y=4+x; let z=7; 3+x*y'
#programstr = '3+5'
programstr = 'let x=3; x'
ast = parser.parse(programstr)
print(ast)

programstr1 = 'let x=3; let y=4; x+y'
ast1 = parser.parse(programstr1)
print(ast1)

programstr2 = 'let x=2; x*(2+x)'
ast2 = parser.parse(programstr2)
print(ast2)

programstr3 = '2+3'
ast3 = parser.parse(programstr3)
print(ast3)

programstr4 = '4*7'
ast4 = parser.parse(programstr4)
print(ast4)

programstr5 = 'x+2'
ast5 = parser.parse(programstr5)
print(ast5)

# OK, you've got the parse tree, now evaluate it!
# potential expr tuples:
#   ('number', 3)
#   ('name', x)
#   ('binop', '+', expr[2], expr[3])
#   ('binop', '-', expr[2], expr[3])
#   ('binop', '*', expr[2], expr[3])
#   ('binop', '/', expr[2], expr[3])
#   ('unary', '-', expr[2])
#   ('unary', '+', expr[2])
#   ('grouped',expr)
def evaluateExpression(expr, symboltable):
  print(expr)
  if expr[0] == 'grouped':
    return evaluateExpression(expr[1], symboltable)
  elif expr[0] == 'name':
    return symboltable[expr[1]]
  elif expr[0] == 'number':
    return 3
  elif expr[0] == 'unary':
    if expr[1] == '-':
      # bug fix - this used to have expr[1] in the recursive call ... updated to expr[2]
      return -evaluateExpression(expr[2], symboltable)
    else:
      return evaluateExpression(expr[2], symboltable)
  elif expr[0] == 'binop':
    if expr[1] == '+':
      return evaluateExpression((expr[2], symboltable) + (expr[3], symboltable)) 
    elif expr[1] == '-':
      return evaluateExpression((expr[2], symboltable) - (expr[3], symboltable))
    elif expr[1] == '*':
      return evaluateExpression((expr[2], symboltable) * (expr[3], symboltable))
    else:
      return evaluateExpression((expr[2], symboltable) / (expr[3], symboltable))
  else:    
    return 0

# example symbollist: ('w', ('number', 5), 'x', ('number', 3), 'y', ('binop', '+', ('number', 4), ('name', 'x')), 'z', ('number', 7))
def populateSymbols(symbollist):
  symboltable = {}
  for i in range(len(symbollist)):
    if i%2==0:
      symboltable[symbollist[i]] = evaluateExpression(symbollist[i+1], symboltable)
  return symboltable

def evaluate(ast):
  if ast[0]=='assignments':
    return evaluateExpression(ast[2], populateSymbols(ast[1]))
  else:
    return evaluateExpression(ast, {})

/content/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply/ply
--2023-02-24 17:28:36--  https://raw.githubusercontent.com/dabeaz/ply/master/src/ply/lex.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35241 (34K) [text/plain]
Saving to: ‘lex.py’

lex.py              100%[===================>]  34.42K  --.-KB/s    in 0.002s  

2023-02-24 17:28:36 (18.4 MB/s) - ‘lex.py’ saved [35241/35241]

--2023-02-24 17:28:36--  https://raw.githubusercontent.com/dabeaz/ply/master/src/ply/yacc.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... conne

In [ ]:
# at this point it should print 0 based on what I have given you
print(evaluate(ast))

('number', 3)
('name', 'x')
3
